In [ ]:
#!pip install spacy

In [2]:
!#python -m spacy download en_core_web_sm

In [22]:
import spacy
import en_core_web_sm
import json
import random
#from sklearn.model_selection import train_test_split
from spacy.util import minibatch, compounding

In [15]:
json_file = open('training_set.json')
data = json.load(json_file)

In [17]:
y_training = [datapoint["intent"] for datapoint in data]
x_training = [datapoint["sentence"] for datapoint in data]

In [25]:
print(y_training[2], " : ",x_training[2])
print(len(y_training),len(x_training))

purchase  :  Le meilleur cabriolet hybrid moins de 5m10 minimum 400 litres de coffre ?
6035 6035


In [27]:
nlp = en_core_web_sm.load()
# Adding the built-in textcat component to the pipeline.
textcat=nlp.create_pipe( "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})
nlp.add_pipe(textcat, last=True)
nlp.pipe_names

['tagger', 'parser', 'ner', 'textcat']

In [6]:
textcat.add_label("find-train")
textcat.add_label("irrelevant")
textcat.add_label("find-flight")
textcat.add_label("find-restaurant")
textcat.add_label("purchase")
textcat.add_label("find-around-me")
textcat.add_label("provide-showtimes")
textcat.add_label("find-hotel")

1

In [ ]:
train_data = list(zip(train_texts,[{'cats': cats} for cats in train_cats]))

In [ ]:
# Disabling other components
n_iter=10


other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()

    print("Training the model...")
    print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F'))

    # Performing training
    for i in range(n_iter):
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)

      # Calling the evaluate() function and printing the scores
        with textcat.model.use_params(optimizer.averages):
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}'  
              .format(losses['textcat'], scores['textcat_p'],
                      scores['textcat_r'], scores['textcat_f']))

In [12]:
%cd Wiidi
!python -m spacy debug-data en ./training_set.json ./testing_set.json

[Errno 2] No such file or directory: 'Wiidi'
/home/jovyan/Wiidi

=========================== Data format validation ===========================

Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.8/site-packages/spacy/__main__.py", line 33, in <module>
    plac.call(commands[command], sys.argv[1:])
  File "/opt/conda/lib/python3.8/site-packages/plac_core.py", line 367, in call
    cmd, result = parser.consume(arglist)
  File "/opt/conda/lib/python3.8/site-packages/plac_core.py", line 232, in consume
    return cmd, self.func(*(args + varargs + extraopts), **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/spacy/cli/debug_data.py", line 85, in debug_data
    corpus = GoldCorpus(train_path, dev_path)
  File "gold.pyx", line 174, in spacy.gold.GoldCorpus